In [1]:
from library.quantum_field import *
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import plotly.io as pio
import time
template="plotly_dark"
np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

#### Particles

In [20]:
# Construct the slits
wallMass = 1
yWall = 0
slitAperture = 30
leftEdge = Object(-slitAperture / 2, yWall, wallMass)
leftWall = [Object(-100 - slitAperture / 2, yWall, wallMass), leftEdge]
rightEdge = Object(slitAperture / 2, yWall, wallMass)
rightWall = [Object(100 + slitAperture / 2, yWall, wallMass), rightEdge]
edges = [leftEdge, rightEdge]
walls = [leftWall, rightWall]

# Comment out for slits
# edges = [rightEdge]
# walls = [rightWall]


# Create the particles
v_y_particles = 300_000
particle_y0_step = 1
particle_x0_fraction = 100  # 100

particles = [
    Particle(x_t0=i / particle_x0_fraction, y_t0=j, v_x=0, v_y=v_y_particles, mass=1)
    for j in range(0, 1, particle_y0_step)
    for i in range(
        -(slitAperture // 2 - wallMass) * particle_x0_fraction,
        (slitAperture // 2 - wallMass) * particle_x0_fraction,
        1,
    )
]


# Define parameters
dt = 0.5  # 0.5
cos_sin = "cos"  # sin
power = 1
r_modifier = 10  # 10
G = 5_000_000
c = 0
gravityCutoff = 1000
yGrid = True
# Sets the limits for the x axis in [-xCutoff, xCutoff]
xCutoff = 50
Y_RANGE = 200

# Punch it
for t in range(Y_RANGE):
    for particle in particles:
        if particle.y > 500:
            dt = 1
        if particle.v_y > 0:
            updateParticle(
                particle,
                edges,
                G=G,
                dt=dt,
                r_modifier=r_modifier,
                c=c,
                power=power,
                gravityCutoff=gravityCutoff,
                cos_sin=cos_sin,
                yGrid=yGrid,
            )


In [21]:
grid = pd.DataFrame(np.vstack((np.hstack([particle.x_trail for particle in particles]), np.hstack([particle.y_trail for particle in particles])))).T
grid.rename(columns={0: "x", 1: "y"}, inplace=True)
grid["x"] = np.round(grid.x, 0)
grid_pt = grid[(grid.x > -xCutoff) & (grid.x < xCutoff) & (grid.y > yWall)].pivot_table(index="y", columns="x", aggfunc=len, fill_value=0)
grid_np = grid_pt.to_numpy()

#### Histogram3D

In [22]:
fig = go.Figure()

fig.add_trace(
    go.Surface(
        z=grid_np,
        x=grid_pt.columns.tolist(),
        y=grid_pt.index.tolist(),
        showscale=False,
        hoverinfo=None,
        reversescale=True,
        name="test",
        colorscale=["red", "black"],
        # lighting=dict(diffuse=0.2, specular=0.0, fresnel=0.1, roughness=0.1),
#         cmin=0.5,
#         cmax=10,
        contours_z=dict(usecolormap=False, project_z=False),
        contours_y=dict(highlightcolor="red", project_y=True),
        contours_x=dict(highlightcolor="red", project_x=True),
    )
)
fig.update_layout(template=template)
fig.update_layout(
    scene=dict(
        xaxis_showspikes=False,
        yaxis_showspikes=False,
        zaxis_showspikes=False,
        xaxis_visible=False,
        yaxis_visible=False,
        zaxis_visible=False,
        yaxis_range=[grid_pt.index.to_numpy().min(), grid_pt.index.to_numpy().max() *1.5]
    ),
    margin=dict(t=0, b=0, l=0, r=0),
)
fig.write_html(f"plotly_graphs/3d_histogram_test{time.time()}.html")
# fig.show()


In [6]:
fig = px.histogram(grid[grid.y == 30], x="x", template=template, nbins=500, histnorm="probability")
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.write_html(f"plotly_graphs/histogram1_test_{time.time()}.html")
# fig.show()


#### Field

In [7]:
field_x = [grid.x.min(), grid.x.max()]
field_y = [grid.y.min(), grid.y.max()]
mesh_size_x = int(np.abs(field_x[0]) + np.abs(field_x[1]))
mesh_size_y = int(np.abs(field_y[0]) + np.abs(field_y[1]))

x = np.linspace(field_x[0], field_x[1], mesh_size_x+1)
y = np.linspace(field_y[0], field_y[1], mesh_size_y+1)

X, Y = np.meshgrid(x, y)

zarray = np.zeros((x.size, y.size))
edges = [-slitAperture/2, slitAperture/2]
param = {
    "mult": -wallMass * G,
    "v": v_y_particles,
    "power": power,
    "trig": cos_sin,
    "wave_length": r_modifier,
}
vmin = -0.001
vmax = 0.01

zarray = f(X, Y, edges=edges, **param)

zarray[zarray < vmin] = vmin
zarray[zarray > vmax] = vmax
zarray = np.nan_to_num(zarray, False, 0)

#### TimeScatter

In [8]:
grid_plot = grid
grid_plot["y_"] = 0
fig = px.scatter(grid, x="x", y="y_", animation_frame="y", animation_group="x", template=template, opacity=0.1, color_discrete_sequence=["crimson"])
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.write_html(f"plotly_graphs/test_timescatter{time.time()}.html")


#### Scatter3d

In [9]:
rightWall = [Object(field_y[1] + slitAperture / 2, yWall, wallMass), rightEdge]
fig = go.Figure(
    data=[
        go.Surface(x=x, y=y, z=zarray, showscale=False, hoverinfo=None, name="2slits")
    ]
    + [
        go.Scatter3d(
            x=particle.x_trail,
            y=particle.y_trail,
            z=[0.01 for point in particle.x_trail],
            mode="lines",
            marker_size=1,
            marker_color="crimson",
            opacity=0.1,
            line_width=5,
        )
        for particle in particles
    ]
    # + [
    #     go.Scatter3d(
    #         x=[particle.x for particle in particles],
    #         y=[particle.y for particle in particles],
    #         z=[0 for particle in particles],
    #         mode="markers",
    #         marker_color="red",
    #         marker_size=2,
    #         line_width=1,
    #     )
    # ]
    + [
        go.Scatter3d(
            x=[edge.x for edge in wall],
            y=[edge.y for edge in wall],
            z=[0.01 for edge in wall],
            mode="markers+lines",
            marker_color="darkgray",
        )
        for wall in walls
    ],
    layout=dict(width=1600, height=800),
)
fig.for_each_trace(
    lambda trace: trace.update(
        colorscale=["red", "grey"],
        reversescale=False,
        cmax=0,
        cmin=-1,
        contours_z=dict(usecolormap=False, project_z=False),
        contours_y=dict(highlightcolor="red", project_y=True),
    )
    if trace.name == "2slits"
    else (),
)
fig.update_layout(template=template)
fig.update_layout(
    scene=dict(
        xaxis_showspikes=False,
        yaxis_showspikes=False,
        zaxis_showspikes=False,
        xaxis_visible=False,
        yaxis_visible=False,
        zaxis_visible=False,
    ),
    margin=dict(t=0, b=0, l=0, r=0),
)
fig.write_html(f"plotly_graphs/scatter3d_test_{time.time()}.html")
# fig.show()